In [1]:
from langchain_groq import ChatGroq
import os

In [2]:
llm = ChatGroq(
    temperature=0,
    groq_api_key=os.getenv('GROQ_API_KEY'),  # Using environment variable for the API key
    model_name='llama-3.1-70b-versatile'
)
response = llm.invoke("The first person to land on moon was")

## WebBaseLoader class of the LangChain module 
A web-scraper 

In [3]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-36059?from=job%20search%20funnel")
page_data = loader.load().pop().page_content 
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Apply for Nike Factory Store - Accounting/Administrative Assistant - Ontario, CA

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsRetail StoresNike Factory Store - Accounting/Administrat

In [6]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the career's page of a website.
    Your job is to extract the job postings and return them in JSON format containing the keys:
    'role', 'experience level', 'skills', and 'description'
    Only return the valid JSON.
    ###VALID JSON (NO PREAMBLE):

"""
)


chain_extract = prompt_extract | llm # This is the LangChain chain - you have a prompt and you are passing it to LLM
response = chain_extract.invoke(input = {"page_data": page_data}) # This is the response from the chain
print(response.content)

```json
{
  "role": "Accounting/Administrative Assistant",
  "experience level": "1-year minimum retail experience",
  "skills": [
    "Proficient knowledge of office practices, procedures, and equipment",
    "Intermediate skills in Microsoft Office products including Word and Excel",
    "Ability to use the Internet/Intranet as a resource for department work activities",
    "Strong customer service skills",
    "Strong attention to detail and deadlines"
  ],
  "description": "Administer advanced cashiering tasks including all cash-related opening/closing duties, assembling/verifying cash deposits, counting tills, and ensuring that cashiers have adequate change on hand. Support scheduling through zone chart report generation and identifying/aiding with opportunities in daily athlete coverage. Review athlete tech functionality and store facilities to help troubleshoot, submit service tickets, and track ticket statuses. Ability to perform standard cashiering duties and resolve irregula

The LLM has extracted all the key desired information from the given content. This is the power of LLM. 

In [7]:
type(response.content)

str

Note that the output is a string. We have to convert it into a json (viz. a dictionary) format using LangChain JSON Parser class.

In [10]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_response = json_parser.parse(response.content)
json_response

{'role': 'Accounting/Administrative Assistant',
 'experience level': '1-year minimum retail experience',
 'skills': ['Proficient knowledge of office practices, procedures, and equipment',
  'Intermediate skills in Microsoft Office products including Word and Excel',
  'Ability to use the Internet/Intranet as a resource for department work activities',
  'Strong customer service skills',
  'Strong attention to detail and deadlines'],
 'description': 'Administer advanced cashiering tasks including all cash-related opening/closing duties, assembling/verifying cash deposits, counting tills, and ensuring that cashiers have adequate change on hand. Support scheduling through zone chart report generation and identifying/aiding with opportunities in daily athlete coverage. Review athlete tech functionality and store facilities to help troubleshoot, submit service tickets, and track ticket statuses. Ability to perform standard cashiering duties and resolve irregular or difficult transactions. P

In [11]:
type(json_response)

dict

Now, a dummy file named 'my_portfolio.csv' has been created that contains the content of the portfolio - Various technologies that the company has human resources for and the corresponding portfolio links.

The LLM should be able to match the one or more technologies from the portfolio with the content of the job description and provide the corresponding portfolio links.

In [12]:
import pandas as pd

df = pd.read_csv('my_portfolio.csv')
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


This data has to be sent to chromadb database for further processing.

In [14]:
import chromadb
import uuid

client  = chromadb.PersistentClient('vectorstore') #.Client  creates a new client each time it is called, while PersistentClient creates a client that can be reused (by storing it on the disk)
# Thus, PersistentClient will generate a local database that can be reused for future use
collection = client.get_or_create_collection(name='portfolio')

if not collection.count(): # If the collection is empty, then add the data
    for _, row in df.iterrows():
        collection.add(documents=row['Techstack'],
                       metadatas={"links": row['Links']},
                       ids = [str(uuid.uuid4())])
        
# A folder will be generated by the name of 'vectorstore' in the current directory where the database will be stored.

In [16]:
# Trial query
links = collection.query(query_texts=["Experience in Python", "Experience in React"], n_results=2)
links

{'ids': [['e756e802-d653-4052-a39f-b0c2821431d8',
   'f0e9dcc9-a6d1-4305-b9ba-d7a1c122a4c9'],
  ['2d7dbbed-55b4-4bc5-82cf-9867f5062899',
   '0b87744e-0512-4db0-b7f9-6851712f0e79']],
 'distances': [[0.9968848007555503, 1.0578701721493369],
  [0.9630732819680132, 1.0685748833873279]],
 'metadatas': [[{'links': 'https://example.com/ml-python-portfolio'},
   {'links': 'https://example.com/python-portfolio'}],
  [{'links': 'https://example.com/react-portfolio'},
   {'links': 'https://example.com/react-native-portfolio'}]],
 'embeddings': None,
 'documents': [['Machine Learning, Python, TensorFlow',
   'Python, Django, MySQL'],
  ['React, Node.js, MongoDB', 'React Native, Node.js, MongoDB']],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents', 'distances']}

In [17]:
collection.query(query_texts=["Experience in Python", "Experience in React"], n_results=2).get('documents')

[['Machine Learning, Python, TensorFlow', 'Python, Django, MySQL'],
 ['React, Node.js, MongoDB', 'React Native, Node.js, MongoDB']]

For the first one, it is bringing ['Machine Learning, Python, TensorFlow','Python, Django, MySQL'] and for the second one ['React, Node.js, MongoDB', 'React Native, Node.js, MongoDB'].

In [19]:
job = json_response
job["skills"]

['Proficient knowledge of office practices, procedures, and equipment',
 'Intermediate skills in Microsoft Office products including Word and Excel',
 'Ability to use the Internet/Intranet as a resource for department work activities',
 'Strong customer service skills',
 'Strong attention to detail and deadlines']

In [20]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/wordpress-portfolio'},
  {'links': 'https://example.com/xamarin-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/kotlin-android-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ios-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}]]

# Generate Email

In [21]:
prompt_email = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
    {job_description}

    ### INSTRUCTION:
    You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
    the seamless integration of business processes through automated tools. 
    Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
    process optimization, cost reduction, and heightened overall efficiency. 
    Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
    in fulfilling their needs.
    Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
    Remember you are Mohan, BDE at AtliQ. 
    Do not provide a preamble.
    ### EMAIL (NO PREAMBLE):
"""
)

chain_email = prompt_email | llm
response = chain_email.invoke(input = {"job_description": str(job), "link_list": links})
print(response.content)


Subject: Streamlining Accounting and Administrative Tasks with AtliQ's Expertise

Dear Hiring Manager,

I came across the job description for an Accounting/Administrative Assistant at your esteemed organization and was impressed by the role's requirements. As a Business Development Executive at AtliQ, I believe our company can help you streamline and automate various tasks mentioned in the job description, thereby increasing efficiency and reducing costs.

Our team at AtliQ has extensive experience in developing customized software solutions that cater to the specific needs of businesses like yours. We can help you automate tasks such as cash-related opening/closing duties, assembling/verifying cash deposits, and generating zone chart reports. Our expertise in Microsoft Office products, including Word and Excel, can also be leveraged to create tailored solutions for your organization.

Our portfolio showcases our capabilities in developing scalable and efficient solutions. Some relevan